In [52]:
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, SelectFwe, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.decomposition import PCA
from concurrent.futures import ProcessPoolExecutor
import pandas as pd
import numpy as np
import warnings

In [48]:
def produce_modifications(X_train, y_train, train_indices, target_indices, X_val, y_val_na):
    for i in target_indices:

        if i in train_indices:
            mod0 = np.copy(y_train)
            mod0[i] = 1 - mod0[i]
            yield X_train, mod0, train_indices, X_val, y_val_na

            mod1 = list(train_indices)
            mod1.remove(i)
            yield X_train, y_train, mod1, X_val, y_val_na

        else:
            mod0 = list(train_indices)
            mod0.append(i)
            yield X_train, y_train, mod0, X_val, y_val_na

            mod1 = np.copy(y_train)
            mod1[i] = 1 - mod1[i]
            yield X_train, mod1, mod0, X_val, y_val_na

def test_modification(test):
    X_train, y_train, train_indices, X_val, y_val_na = test
    
    clf = LinearRegression(copy_X=True)
    clf.fit(X_train[train_indices],y_train[train_indices])
    new_error = clf.residues_
    
    return new_error, y_train, train_indices

def ce_squared(T, probs):
    return ((T*probs)**2).sum()/len(probs)

def process_dataset(X_train, y_train, X_test):
    # Select top 50 features for mutual info with y
    minfos = mutual_info_classif(X_train, y_train) 
    minds = minfos.argsort()[-50:]
    X_train = X_train[:, minds]
    
    # Map it down to 30 dims while preserving ~99% of variance
    pca = PCA(n_components=30) 
    X_train = pca.fit_transform(X_train)
    
    # Transform X_test
    X_test = pca.transform(X_test[:,minds])
    np.random.shuffle(X_test)
    
    # Undersample & seperate a validation set
    pinds = np.where(y_train == 1)[0] 
    ninds = np.where(y_train == 0)[0]
    
    np.random.shuffle(ninds)
    
    train_ninds = ninds[:500]
    val_ninds = ninds[-300:]
    
    train_pinds = pinds[:300]
    val_pinds = pinds[-100:]
    
    X_tr = np.vstack((X_train[train_pinds], X_train[train_ninds]))
    y_tr = np.append(y_train[train_pinds], y_train[train_ninds])
    
    X_va = np.vstack((X_train[val_pinds], X_train[val_ninds]))
    y_va = np.append(y_train[val_pinds], y_train[val_ninds])
    
    tr_inds = list(range(X_tr.shape[0]))
    np.random.shuffle(tr_inds)
    
    X_train = X_tr[tr_inds]
    y_train = y_tr[tr_inds]
    
    va_inds = list(range(X_va.shape[0]))
    np.random.shuffle(va_inds)
    
    X_val = X_va[va_inds]
    y_val = y_va[va_inds]
    
    return X_train, y_train, X_val, y_val, X_test    

def modify_dataset(X_train, y_train, X_val, y_val):
    start_ind = 0
    batch_size = 100
    end_ind = start_ind + batch_size

    y_val_na = y_val[:, np.newaxis]
    y_val_na = np.append(y_val_na, 1-y_val_na, axis=1)
    
    clf = LinearRegression(copy_X=True)
    clf.fit(X_train, y_train)
    
    best_error = clf.residues_
    best_y_train = y_train
    best_train_indices = list(range(X_train.shape[0]))

    while end_ind <= X_train.shape[0]:
        target_indices = range(start_ind, end_ind)
        mods = produce_modifications(X_train, best_y_train, best_train_indices, target_indices, X_val, y_val_na)

        test_results = list(map(test_modification, mods))
        test_results.append((best_error, best_y_train, best_train_indices))
        best_error, best_y_train, best_train_indices = min(test_results, key=lambda x: x[0])

        print('Processed: {:5d} samples,\tcurrent error is {:0.4f}'.format(end_ind, best_error))
        start_ind += batch_size
        end_ind += batch_size
        
    return X_train[best_train_indices], best_y_train[best_train_indices]

In [41]:
X_train = np.array(pd.read_csv('train.csv', header=None))
y_train = np.array(pd.read_csv('trainlabels.csv', header=None)).ravel()
X_test = np.array(pd.read_csv('test.csv', header=None))

X_train, y_train, X_val, y_val, X_test = process_dataset(X_train, y_train, X_test)

In [49]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    warnings.warn("deprecated", DeprecationWarning)
    X, y = modify_dataset(X_train, y_train, X_val, y_val)

Processed:   100 samples,	current error is 74.8882
Processed:   200 samples,	current error is 74.2771
Processed:   300 samples,	current error is 73.7341
Processed:   400 samples,	current error is 70.0449
Processed:   500 samples,	current error is 69.4833
Processed:   600 samples,	current error is 68.9168
Processed:   700 samples,	current error is 68.1518
Processed:   800 samples,	current error is 67.4755


In [54]:
X = np.vstack((X, X_val))
y = np.append(y, y_val)

svm = SVC(class_weight='balanced', cache_size=1000)
auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)
svm.fit(X, y)

SVC(C=1.0, cache_size=1000, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [65]:
c = 10 ** np.arange(-3, 3).astype('float')
g = 10 ** np.arange(-3, 3).astype('float')
params = {'C':c, 'gamma':g}

clf = GridSearchCV(svm, params, scoring=auc)
clf.fit(X, y)
clf.best_params_

{'C': 10.0, 'gamma': 0.10000000000000001}

In [67]:
c =  np.linspace(5, 15, 10).astype('float')
g = np.linspace(0.05, 0.15, 10).astype('float')
params = {'C':c, 'gamma':g}

clf = GridSearchCV(svm, params, scoring=auc)
clf.fit(X, y)
clf.best_params_

{'C': 5.0, 'gamma': 0.1388888888888889}

In [68]:
preds = clf.decision_function(X_test)
preds = pd.DataFrame(preds)
preds.index += 1
preds.to_csv('out.csv', index_label='Id', header=['Prediction'])